In [9]:
import pandas as pd
import numpy as np
import math, sys, logging
logging.basicConfig()
log = logging.getLogger('dynamic')
log.setLevel(logging.INFO)

## Shortest common supersequence

Ok, so I was confused here, I thought a supersequence was one that directly contained its subsequences intact, so like how "ABCDE" contains "ABC" and "CDE". But the definition of subsequence also includes "ACE" as a subsequence, because it contains the same elements, the same number of times, in the same order. This is the difference between subsequences and substrings.

In [146]:
a = {0: ('a', 'b'), 1: ('c', 'd')}
print(a[1])
df = pd.DataFrame(a)
print(df)
ssl = pd.DataFrame(-1, index=range(6), columns=range(5), dtype=int)
ssl
ssl.rank(1)
ssl.iat[3,4] = 1
ssl[:] = -1
ssl

sss = 'abcd'
print(sss[1:3])

('c', 'd')
   0  1
0  a  c
1  b  d
bc


In [125]:
def shortest_superseq(s1, s2):
    l1, l2 = len(s1), len(s2)
    seqs = pd.DataFrame(-1,
        index=range(l1+1), columns=range(l2+1),
        dtype=int)
    _ss(s1, s2, l1, l2, seqs)
    return seqs
    
def _ss(s1, s2, i, j, seqs):
    prev = seqs.iat[i,j]
    if prev != -1:
        return prev
    if i == 0:
        ret = j
    elif j == 0:
        ret = i
    elif s1[i-1] == s2[j-1]:
        ret = _ss(s1, s2, i-1, j-1, seqs) + 1
    else:
        r1 = _ss(s1, s2, i-1, j, seqs) + 1
        r2 = _ss(s1, s2, i, j-1, seqs) + 1
        ret = min(r1, r2)
    seqs.iat[i,j] = ret
    return ret
        

In [147]:
def find_supersequences(seqs, s1, s2):
    l1, l2 = len(s1), len(s2)
    sl = []
    msl = seqs.iat[l1, l2]
    _fss(seqs, s1, s2, l1, l2, sl, '')
    return sl
    
def _fss(seqs, s1, s2, i, j, sl, s):
    p = seqs.iat[i,j]
    if i == 0:
        sl.append(s2[0:j]+s)
    elif j == 0:
        sl.append(s1[0:i]+s)
    else:
        if s1[i-1] == s2[j-1]:
            _fss(seqs, s1, s2, i-1, j-1, sl, s1[i-1]+s)
        else:
            if seqs.iat[i,j] - 1 == seqs.iat[i-1,j]:
                _fss(seqs, s1, s2, i-1, j, sl, s1[i-1]+s)
            if seqs.iat[i,j] - 1 == seqs.iat[i,j-1]:
                _fss(seqs, s1, s2, i, j-1, sl, s2[j-1]+s)
    

In [148]:
s1 = 'ABCBDAB'
s2 = 'BDCABA'
seqs = shortest_superseq(s1, s2)
seqslabeled = seqs.copy()
seqslabeled.loc[1:,('str')] = list(s1)
seqslabeled = seqslabeled.T
seqslabeled.loc[1:-1,('str')] = list(s2)
print(seqslabeled)
seqlist = find_supersequences(seqs, s1, s2)
seqlist


       0   1   2   3   4   5   6   7  str
0     -1   1  -1   3  -1  -1  -1  -1  NaN
1      1   2   2   3   4  -1  -1  -1    B
2      2   3   3   4   5   5  -1  -1    D
3      3   4   4   4   5   6  -1  -1    C
4     -1   4   5   5   6   7   7  -1    A
5     -1  -1  -1  -1   6   7  -1   8    B
6     -1  -1  -1  -1  -1  -1   8   9    A
str  NaN   A   B   C   B   D   A   B  NaN


['ABDCABDAB', 'ABDCBDABA', 'ABCBDCABA']